!pip install -U numpy

import sys
print(sys.path) 
print(sys.executable)

import sys 
!{sys.executable} -m pip install nltk

# Import Packages

In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")


# Dataset Cleaning

In [ ]:
path = '/home/manikya_varshney/Documents/Python/Yale/Data/h01-20200912-101538.csv'
data = pd.read_csv(path, low_memory=False)

In [ ]:
data.shape

In [ ]:
cols = [1, 3, 4, 5,6, 7, 8, 9, 10, 11, 17, 18, 19, 22, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 54, 55, 56, 58, 63, 64, 65, 67, 76, 77]

In [ ]:
data = data[data.columns.values[cols]]

In [ ]:
data.shape

In [ ]:
data.to_csv('data_filtered_h01-20200912-101538.csv',index=False)

In [ ]:
data['lang'].value_counts()

In [ ]:
data[data['lang'] == 'en'].shape

In [ ]:
data_en = data[data['lang'] == 'en'].reset_index(drop=True)

In [ ]:
data_en

In [ ]:
data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'

In [ ]:
data_en[np.where((data_en['is_quote_tweet']==False) & (data_en['RT_Flag']==False),True,False)].reset_index(drop=True)

# Analysis

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
def load_dataset(path, cols):
    df = pd.read_csv(path, low_memory=False)
    df.coloumns = cols
    return df

In [ ]:
def delete_cols(df, cols):
    for col in cols